In [1]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_0/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [2]:
# !ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_0/*nonpeaks_negatives.bed | head

In [3]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_1/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [4]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_2/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [5]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_3/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [6]:
!ls -lt $GROUP_SCRATCH/eila/encode_pseudobulks/encode_pseudobulks_negative/*/*/*/fold_4/*nonpeaks_negatives.bed | wc -l
#1354


1354


In [5]:
!squeue --user=$USER | wc -l

# 8


3


In [86]:
!squeue --user=$USER

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
56469636_[1-381%38 akundaje, step62.b     eila PD       0:00      1 (Priority)
56469701_[1-973%97 akundaje, 633_moti     eila PD       0:00      1 (Priority)


In [82]:
!scancel 56439941

In [3]:
# !rm ./local_logs/633*
# !./execute_sbatch_arrays_on_sherlock.sh ./steps_inputs/step6_3/debug_one_modisco_results_profile_scores_h5.txt step6-3-3-qc-bias-tn5.sh 1 1 


In [4]:
!grep "qval" ./local_logs/slurm.step62.bpnetPipeline.combined.out

In [15]:
# !wc -l ./steps_inputs/step6/chrombpnet_pipeline_extracted_paths.txt
!head -5 ./local_logs/slurm.step62.bpnetPipeline.combined.out

Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR110NNR/ENCFF280HZH/fold_1/step62.bpnetPipeline. Exiting.
Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR110JCS/ENCFF691JXI/fold_3/step62.bpnetPipeline. Exiting.
Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR110NNR/ENCFF280HZH/fold_2/step62.bpnetPipeline. Exiting.
Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR110NNR/ENCFF280HZH/fold_3/step62.bpnetPipeline. Exiting.
Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR110NNR/ENCFF280HZH/fold_4/step62.bpne

In [6]:
!tail -5 ./local_logs/slurm.step62.bpnetPipeline.combined.out

Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR607ITW/ENCFF502THJ/fold_4/step62.bpnetPipeline. Exiting.


In [7]:
!grep modisco ./local_logs/slurm.step62.bpnetPipeline.combined.err

In [8]:
!grep "qval" ./local_logs/slurm.step62.bpnetPipeline.combined.err

In [9]:
!head -5 ./local_logs/slurm.step62.bpnetPipeline.combined.err

++ conda shell.bash hook
+ eval 'export CONDA_EXE='\''/home/users/eila/miniconda3/bin/conda'\''
export _CE_M='\'''\''
export _CE_CONDA='\'''\''
export CONDA_PYTHON_EXE='\''/home/users/eila/miniconda3/bin/python'\''


In [10]:
!tail -5 ./local_logs/slurm.step62.bpnetPipeline.combined.err

+ for folder in '"${REQUIRED_FOLDERS[@]}"'
+ '[' '!' -d /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR607ITW/ENCFF502THJ/fold_4/step62.bpnetPipeline/models ']'
+ true
+ echo 'Output directory and required folders already exist: /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR607ITW/ENCFF502THJ/fold_4/step62.bpnetPipeline. Exiting.'
+ exit 0


In [6]:
# !scancel 56235764
# !scancel $USER
# !ls ./steps_inputs/step6/

In [30]:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/fold_0/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 
#972


972


In [31]:

# fold 1:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/fold_1/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 
# 709



709


In [32]:

# fold 2:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/fold_2/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 
# 474



474


In [33]:


# fold 3:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/fold_3/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 
# 354



354


In [34]:


# fold 4:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/fold_4/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 

# 377


377


In [35]:


# all folds:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/*/step62.bpnetPipeline/evaluation/overall_report.html | wc -l 
# 2886



2886


In [58]:
# 5416/2134

In [42]:
# !scancel -u $USER

In [9]:
# !squeue -u $USER -o "%i %j" | grep step5Neg


In [44]:
 # scp -r eila@login.sherlock.stanford.edu:/scratch/users/eila/encode_pseudobulks_model_training/human/ENCSR037JDN/ENCFF933KCP/fold_1_30000_20240912_182023/evaluation . 

# !rm ./local_logs/slurm_samools_err.combined.err
# !rm ./local_logs/slurm_samools_out.combined.out

In [1]:
!ls execute*

execute_sbatch_arrays_on_sherlock-Copy1.sh
execute_sbatch_arrays_on_sherlock-Copy2.sh
execute_sbatch_arrays_on_sherlock.sh


In [25]:
%%bash
find /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/*/*/fold_0/step62.bpnetPipeline/evaluation/ -name "overall_report.html" \
    | while read dir; do
        # Get the parent directory of "evaluation"
        parent_dir=$(dirname "$(dirname "$dir")")
        
        # Check if modisco_results_profile_scores.h5 exists under auxiliary/interpret_subsample
        if ! find "$parent_dir/auxiliary/interpret_subsample" -name "modisco_results_profile_scores.h5" > /dev/null; then
            # Print the directory containing overall_report.html if modisco_results_profile_scores.h5 is not found
            echo "$parent_dir"
        fi
    done





In [32]:
import os
import glob

# Set the base directories for the two files
evaluation_dir = '/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/*/*/fold_0/step62.bpnetPipeline/evaluation'
modisco_dir = '/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/*/*/fold_0/step62.bpnetPipeline/auxiliary/interpret_subsample'

# Find all directories with the relevant files
overall_report_dirs = set(os.path.dirname(path) for path in glob.glob(f"{evaluation_dir}/overall_report.html"))
modisco_dirs = set(os.path.dirname(path) for path in glob.glob(f"{modisco_dir}/modisco_results_profile_scores.h5"))

# Get the base path up to fold_0/step62.bpnetPipeline/
base_path = '/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/*/*/fold_0/step62.bpnetPipeline'

# Find directories missing at least one file
missing_files_dirs = set()

# Iterate over directories up to the base path level
for dir in glob.glob(f"{base_path}/*/*/"):
    # If the directory has overall_report.html but not modisco_results_profile_scores.h5
    if dir in overall_report_dirs and dir not in modisco_dirs:
        missing_files_dirs.add(dir)
    # If the directory has modisco_results_profile_scores.h5 but not overall_report.html
    elif dir in modisco_dirs and dir not in overall_report_dirs:
        missing_files_dirs.add(dir)

# Print out the result
print(f"Directories missing at least one of the files ('overall_report.html' or 'modisco_results_profile_scores.h5'):")
for dir in missing_files_dirs:
    print(dir)


Directories missing at least one of the files ('overall_report.html' or 'modisco_results_profile_scores.h5'):


In [28]:
import os

def find_folders_without_file(base_path, target_file):
    # Walk through the directory structure
    folders_without_file = []

    for root, dirs, files in os.walk(base_path):
        # Check if the current directory contains the target file
        if target_file not in files:
            folders_without_file.append(root)
    
    return folders_without_file

# Define the base path and the target file
base_path = '/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/*/*/*/fold_0/'
target_file = 'step62.bpnetPipeline/auxiliary/interpret_subsample/modisco_results_profile_scores.h5'

# Call the function to find folders without the file
folders = find_folders_without_file(base_path, target_file)

# Print the results
for folder in folders:
    print(folder)


In [29]:
# !ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR000NVR/ENCFF585MYS/fold_0/step62.bpnetPipeline/auxiliary/interpret_subsample/modisco_results_profile_scores.h5



In [50]:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_data/bams/ENCSR000NVR



ENCSR000NVR_sorted.bam	ENCSR000NVR_sorted.bam.bai


In [59]:
!ls /scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR000NVR/ENCFF585MYS/fold_0/step62.bpnetPipeline/auxiliary/interpret_subsample/modisco_results_profile_scores.h5




/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training/human/ENCSR000NVR/ENCFF585MYS/fold_0/step62.bpnetPipeline/auxiliary/interpret_subsample/modisco_results_profile_scores.h5


In [ ]:
#  prep for step6-2-chrombpnet-pipelline.sh 
# step 1 : find BAM ID without model

In [76]:
import glob
import os
import csv

def find_bams_ids(base_bams_path):
    """List all BAM IDs from the bams directory."""
    return [d for d in os.listdir(base_bams_path) if os.path.isdir(os.path.join(base_bams_path, d))]

def find_matching_files_with_glob(base_training_path, target_file_pattern, bam_id):
    """Find modisco_results_profile_scores.h5 using glob pattern matching."""
    # Correct the glob pattern
    search_pattern = os.path.join(base_training_path, '*', bam_id, '*', 'fold_0', 'step62.bpnetPipeline', 'auxiliary', 'interpret_subsample', target_file_pattern)
    
    # Print the search pattern for debugging
    print(f"Search pattern for {bam_id}: {search_pattern}")
    
    # Use glob to find the files matching the pattern
    found_paths = glob.glob(search_pattern, recursive=True)
    
    return found_paths, search_pattern

def compare_files(base_bams_path, base_training_path, target_file, output_file):
    """Compare BAM IDs and return those without a matching modisco_results_profile_scores.h5."""
    bams_ids = find_bams_ids(base_bams_path)

    # Open CSV file for writing the results
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Write header
        writer.writerow(["BAM ID", "Found Path", "Destination Path (Expected)", "Match Found?"])
        
        for bam_id in bams_ids:
            # Define the BAM source path
            bam_path = os.path.join(base_bams_path, bam_id)
            
            # Find matching files for the target file using glob
            matching_folders, expected_path = find_matching_files_with_glob(base_training_path, target_file, bam_id)
            
            if matching_folders:
                # If matches found, write the actual folder path where the file was found
                for folder in matching_folders:
                    writer.writerow([bam_id, folder, expected_path, "Yes"])
            else:
                # If no match is found, write the expected search path template
                writer.writerow([bam_id, "", expected_path, "No"])
            
    print(f"Results have been written to {output_file}")

# Define the base paths and target file
base_bams_path = '/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_data/bams'
base_training_path = '/scratch/groups/akundaje/eila/encode_pseudobulks/encode_pseudobulks_model_training'
target_file = 'modisco_results_profile_scores.h5'
output_file = 'missing_models_bams_output.csv'  # Update this to your desired output file location

# Run the comparison and save the output to the file
compare_files(base_bams_path, base_training_path, target_file, output_file)


In [ ]:
# prepare input for the model creation step: step6-2-chrombpnet-pipelline.sh 

In [78]:
import csv

def load_previous_results(previous_results_file):
    """Load previous results to check which BAM IDs did not match."""
    missing_bam_ids = set()  # Set to store BAM IDs with no match
    with open(previous_results_file, mode='r') as file:
        reader = csv.reader(file)
        next(reader)  # Skip header
        for row in reader:
            bam_id = row[0]  # BAM ID is the first column
            match_found = row[3]  # 'Match Found?' is the fourth column
            if match_found == "No":  # We care about rows where "Match Found?" is "No"
                missing_bam_ids.add(bam_id)
    return missing_bam_ids

def find_missing_lines(input_file, missing_bam_ids, output_file):
    """Compare lines in the input file with missing BAM IDs and write relevant ones to the output file."""
    with open(input_file, mode='r') as infile, open(output_file, mode='w') as outfile:
        reader = infile.readlines()
        
        for line in reader:
            # Split the line by space to extract the columns
            columns = line.strip().split()
            bam_id = columns[1]  # BAM ID is in the second column
            
            # Check if the BAM ID is in the missing set
            if bam_id in missing_bam_ids:
                outfile.write(line)  # Write the relevant line to the output file

    print(f"Missing entries have been written to {output_file}")

# Define paths
input_file = './steps_inputs/step6/chrombpnet_pipeline_extracted_paths_fold_0.txt'  # Input text file
previous_results_file = 'missing_models_bams_output.csv'  # Previous results file (CSV)
output_file = './steps_inputs/step6/missing_chrombpnet_pipeline_extracted_paths_fold_0.txt'  # Corrected Output file path

# Load BAM IDs that didn't match from the previous script output
missing_bam_ids = load_previous_results(previous_results_file)

# Compare input file lines with missing BAM IDs and write missing entries to the output file
find_missing_lines(input_file, missing_bam_ids, output_file)


Missing entries have been written to ./steps_inputs/step6/missing_chrombpnet_pipeline_extracted_paths_fold_0.txt
